In [8]:
import pickle
project = 'openstack'
type = 'in_its'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data = pickle.load(f_train)
        te_data = pickle.load(f_test)
pred_list = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+type+'/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list.append(pickle.load(f))
tr_pred = [[pred_list[0][j], pred_list[1][j], pred_list[2][j]] for j in range(len(pred_list[0]))]
te_pred = [[pred_list[3][j], pred_list[4][j], pred_list[5][j]] for j in range(len(pred_list[3]))]                
tr_data = [tr_pred, tr_data[5]]
te_data = [te_pred, te_data[5]]

In [9]:
from optuna.integration import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight

In [10]:
sample_weights = compute_sample_weight(class_weight='balanced', y=tr_data[1])
# _weights = compute_class_weight(class_weight='balanced', classes=np.unique(tr_dataset[5]), y=tr_dataset[5])

In [11]:

lgb_train = lgb.Dataset(np.array(tr_data[0]), np.array(tr_data[1]), weight=sample_weights)

In [12]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
lgb_params = {
    'objective':'binary',
    'metric': 'binary_logloss',
    'verbosity':-1,
}
tuner_cv = lgb.LightGBMTunerCV(
    lgb_params, lgb_train,
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True),
                lgb.log_evaluation(100)],
    return_cvbooster=True,
    folds=folds
)

tuner_cv.run()

[I 2023-12-06 15:53:26,212] A new study created in memory with name: no-name-1505e16a-19e9-48e6-9c6c-a898ca446792
feature_fraction, val_score: 0.527512:  14%|#4        | 1/7 [00:00<00:00,  7.07it/s]

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[25]	cv_agg's valid binary_logloss: 0.527512 + 0.0173715
Training until validation scores don't improve for 10 rounds


feature_fraction, val_score: 0.527512:  43%|####2     | 3/7 [00:00<00:00,  7.02it/s]

Early stopping, best iteration is:
[25]	cv_agg's valid binary_logloss: 0.527512 + 0.0173715
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	cv_agg's valid binary_logloss: 0.533425 + 0.0184439
Training until validation scores don't improve for 10 rounds


feature_fraction, val_score: 0.527512:  71%|#######1  | 5/7 [00:00<00:00,  6.65it/s]

...

Training until validation scores don't improve for 10 rounds
[100]	cv_agg's valid binary_logloss: 0.519485 + 0.0135702
Early stopping, best iteration is:
[90]	cv_agg's valid binary_logloss: 0.519372 + 0.0129857
Training until validation scores don't improve for 10 rounds
[100]	cv_agg's valid binary_logloss: 0.519546 + 0.0136458
Early stopping, best iteration is:
[90]	cv_agg's valid binary_logloss: 0.519396 + 0.012969


[I 2023-12-06 15:53:34,742] Trial 64 finished with value: 0.5193959166075486 and parameters: {'min_child_samples': 10}. Best is trial 63 with value: 0.519372433280849.
min_child_samples, val_score: 0.519372:  60%|######    | 3/5 [00:00<00:00,  9.63it/s]

Training until validation scores don't improve for 10 rounds
[100]	cv_agg's valid binary_logloss: 0.519546 + 0.0136458
Early stopping, best iteration is:
[90]	cv_agg's valid binary_logloss: 0.519396 + 0.012969
Training until validation scores don't improve for 10 rounds
[100]	cv_agg's valid binary_logloss: 0.519493 + 0.013545
Early stopping, best iteration is:
[95]	cv_agg's valid binary_logloss: 0.519391 + 0.0134022


[I 2023-12-06 15:53:34,941] Trial 66 finished with value: 0.519390997659517 and parameters: {'min_child_samples': 100}. Best is trial 63 with value: 0.519372433280849.
min_child_samples, val_score: 0.519155: 100%|##########| 5/5 [00:00<00:00,  9.41it/s]

Training until validation scores don't improve for 10 rounds
[100]	cv_agg's valid binary_logloss: 0.51926 + 0.0132867
Early stopping, best iteration is:
[92]	cv_agg's valid binary_logloss: 0.519155 + 0.013126


In [7]:
#qt logloss 不均衡対策
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
print(tuner_cv.best_params)
model = tuner_cv.get_best_booster()
pred = model.predict(np.array(te_data[0]), num_iteration=model.best_iteration)
mean_pred = np.mean(pred, axis=0)
print(mean_pred)
print(roc_auc_score(te_data[1], mean_pred))
print(log_loss(te_data[1], mean_pred))

Best score: 0.5268616576858833
Best params:
{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 2.083874795538991e-08, 'lambda_l2': 8.069877809166703, 'num_leaves': 5, 'feature_fraction': 1.0, 'bagging_fraction': 0.8534669684662493, 'bagging_freq': 5, 'min_child_samples': 20}
[0.29343171 0.55357103 0.08359059 ... 0.75647117 0.66175313 0.0943424 ]
0.7781673455724762
0.5579963518485113


In [13]:
#openstack logloss 不均衡対策
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
print(tuner_cv.best_params)
model = tuner_cv.get_best_booster()
pred = model.predict(np.array(te_data[0]), num_iteration=model.best_iteration)
mean_pred = np.mean(pred, axis=0)
print(mean_pred)
print(roc_auc_score(te_data[1], mean_pred))
print(log_loss(te_data[1], mean_pred))

Best score: 0.5191551461910671
Best params:
{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 1.0180923859633273e-08, 'lambda_l2': 0.005149005482750249, 'num_leaves': 2, 'feature_fraction': 0.8999999999999999, 'bagging_fraction': 0.8202254509243686, 'bagging_freq': 7, 'min_child_samples': 50}
[0.62849822 0.87283922 0.18856518 ... 0.52197467 0.13077734 0.16327595]
0.8179918935148118
0.6369596997637235


In [69]:
#qt auc 不均衡対策
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
print(tuner_cv.best_params)
model = tuner_cv.get_best_booster()
pred = model.predict(np.array(te_data[0]), num_iteration=model.best_iteration)
mean_pred = np.mean(pred, axis=0)
print(mean_pred)
print(roc_auc_score(te_data[1], mean_pred))
print(log_loss(te_data[1], mean_pred))

Best score: 0.8165328036699602
Best params:
{'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 1.9317001406011016, 'lambda_l2': 3.094407820496894, 'num_leaves': 2, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}
[0.36960416 0.50902352 0.14799592 ... 0.70303968 0.61321876 0.18555734]
0.7655688245292944
0.5847515942649286
